# PrioritizeMe

Aplikacja do priorytetowego planowania zadań to narzędzie, które pomoże efektywnie zarządzać czasem i zadaniami. Dzięki niej możesz tworzyć zadania, przypisywać im tytuły oraz opisy, a następnie nadawać im priorytety w zależności od ich ważności. 

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania wszystkich zadań, umożliwia podstawowe operacje *CRUD* (*Create-Remove-Update-Delete*), oraz ich filtrowanie i sortowanie.

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `LiveData` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania zadań, oraz jako *SSOT* (*Single Source Of Truth*)
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę zadań, umożliwia usunięcie, oznaczenie zadania jako wykonanego, oraz szybką zmianę nadanego priorytetu
    - ekran dodania zadania
    - ekran aktualizacji zadania, który umożliwia również usunięcie
- Aplikacja wykorzystuje `Jetpack Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w głównek aktywności w celu uzyskania współdzielenia jego instancji przez wszystkie fragmenty


Dodajmy wymagane zależności do projektu

```verbatim
>>> Build.gradle(Project)
```

```kotlin
// Top-level build file where you can add configuration options common to all sub-projects/modules.
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath ("androidx.navigation:navigation-safe-args-gradle-plugin:2.5.3")
    }
}
```

```verbatim
>>> Build.gradle(Module)
```

```kotlin
plugins {
    id("com.android.application")
    id("androidx.navigation.safeargs")
}
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}
dependencies {

    implementation ("androidx.navigation:navigation-fragment:2.5.3")
    implementation ("androidx.navigation:navigation-ui:2.5.3")

    // ROOM
    implementation ("androidx.room:room-runtime:2.5.2")
    annotationProcessor ("androidx.room:room-compiler:2.5.2")

// ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel:2.5.1")
// LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata:2.5.1")

    implementation("androidx.cardview:cardview:1.0.0")

    implementation ("androidx.recyclerview:recyclerview:1.3.1")
    ...
}
```

W pierwszym kroku przygotujmy dane, które wykorzystamy jako *dummy data* do inicjalizacji bazy danych.

In [ ]:
@Entity(tableName = "task_table")
public class Task {
    @PrimaryKey(autoGenerate = true)
    private int id = 0;
    private String title;
    private String description;
    private boolean isDone;
    private Priority priority;

    @Ignore // Konstruktor ignorowany przez ROOM
    public Task(String title, String description) {
        this.title = title;
        this.description = description;
        this.isDone = false;
        this.priority = Priority.NORMALNY;
    }

    public Task(String title, String description, boolean isDone, Priority priority) {
        this.title = title;
        this.description = description;
        this.isDone = isDone;
        this.priority = priority;
    }

    @Ignore
    public Task(String title, String description, Priority priority) {
        this.title = title;
        this.description = description;
        this.isDone = false;
        this.priority = priority;
    }

    public int getId() {
        return id;
    }

    public void setId(int id) {
        this.id = id;
    }

    public String getTitle() {
        return title;
    }

    public void setTitle(String title) {
        this.title = title;
    }

    public String getDescription() {
        return description;
    }

    public void setDescription(String description) {
        this.description = description;
    }

    public boolean isDone() {
        return isDone;
    }

    public void setDone(boolean done) {
        isDone = done;
    }

    public Priority getPriority() {
        return priority;
    }

    public void setPriority(Priority priority) {
        this.priority = priority;
    }

    @Override
    public int hashCode() {
        return Objects.hash(id, title, description, isDone, priority);
    }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        Task task = (Task) o;
        return id == task.id &&
                isDone == task.isDone &&
                title.equals(task.title) &&
                description.equals(task.description) &&
                priority == task.priority;
    }
}

In [ ]:
public final class DataProvider {
    private DataProvider(){}

    private static final List<String> titles = Arrays.asList(
            "Dokończ Raport",
            "Przygotuj Prezentację",
            "Zadzwoń do Klienta",
            "Aktualizuj Zawartość Strony",
            "Weź Udział w Spotkaniu Zespołu",
            "Przejrzyj Kod",
            "Wysyłka Faktur",
            "Badanie Trendów na Rynku",
            "Plan Kampanii Marketingowej",
            "Organizuj Pliki",
            "Przygotuj Propozycję Budżetu",
            "Twórz Mockupy Produktu",
            "Planuj Posty na Mediach Społecznościowych",
            "Spotkanie z Zespołem Projektowym",
            "Testuj Nowe Oprogramowanie",
            "Dochodź do Potencjalnych Klientów",
            "Projektuj Logo",
            "Napisz Post na Blogu",
            "Ustal Harmonogram Projektu",
            "Przejrzyj Specyfikacje Produktu",
            "Organizuj Plan Podróży",
            "Przeprowadź Ankietę Użytkowników",
            "Analizuj Dane Sprzedażowe",
            "Przygotuj Materiały Szkoleniowe",
            "Koordynuj Logistykę Wydarzenia",
            "Burza Mózgów nad Nowymi Pomysłami",
            "Optymalizuj SEO Strony",
            "Przeprowadź Oceny Wyników Pracowników",
            "Twórz Prototypy Aplikacji"
    );

    private static final List<String> descriptions = Arrays.asList(
            "Ukończ końcowy raport dotyczący wyników kwartalnych.",
            "Stwórz prezentację PowerPoint na nadchodzące spotkanie z klientem.",
            "Skontaktuj się z klientem w celu dalszej rozmowy.",
            "Aktualizuj treść na stronie głównej witryny internetowej.",
            "Zorganizuj logistykę nadchodzącego wydarzenia firmowego.",
            "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu.",
            "Optymalizuj SEO strony internetowej dla lepszych wyników w wyszukiwarkach.",
            "Przeprowadź oceny wyników pracowników zespołu.",
            "Twórz interaktywne prototypy dla nowej aplikacji.",
            "Dokładnie przetestuj i oceniaj funkcjonalność nowego oprogramowania. Dokumentuj wszelkie błędy lub problemy i współpracuj blisko z zespołem developerskim, aby zapewnić ich szybkie rozwiązanie.",
            "Kontynuuj kontakt z potencjalnymi klientami i odpowiedz na zapytania wygenerowane przez kampanie marketingowe i wydarzenia networkingowe. Udziel spersonalizowanych odpowiedzi i dostarcz informacji dostosowanych do potrzeb każdego potencjalnego klienta.",
            "Zaprojektuj nowe logo dla przemiany marki, biorąc pod uwagę tożsamość marki, grupę docelową i trendy w branży. Przedstaw wiele opcji logo w celu uzyskania opinii i wybierz ostateczny projekt.",
            "Napisz dobrze przemyślany i informacyjny post na blogu na temat najnowszych wydarzeń w branży. Wykorzystaj dane, statystyki i wiedzę ekspertów, aby dostarczyć cenne treści czytelnikom bloga.",
            "Ustal harmonogram dla nadchodzących faz projektu, uwzględniając dostępność zasobów, zależności i potencjalne ryzyka. Podziel się harmonogramem z interesariuszami projektu w celu uzyskania opinii.",
            "Przejrzyj i sfinalizuj specyfikacje produktu, upewniając się, że są one kompleksowe i zgodne z celami projektu. Podziel się specyfikacjami z zespołem developerskim w celu wdrożenia.",
            "Zaplanuj plan podróży służbowej, uwzględniając rozkłady lotów, zakwaterowanie, transport i organizację spotkań. Podziel się planem podróży ze wszystkimi odpowiednimi członkami zespołu.",
            "Przeprowadź szczegółową ankietę, aby pozyskać opinię użytkowników na temat użyteczności produktu, funkcji i ogólnego zadowolenia. Analizuj wyniki ankiety, aby zidentyfikować obszary do poprawy.",
            "Analizuj dane sprzedażowe i identyfikuj trendy w celu podejmowania decyzji opartych na danych dotyczących strategii sprzedaży i poprawek produktowych. Przygotuj obszerny raport, aby przedstawić wyniki zespołowi sprzedażowemu i zarządowi.",
            "Przygotuj materiały szkoleniowe dla nowych pracowników, w tym przewodniki wprowadzające, moduły szkoleniowe i prezentacje. Upewnij się, że materiały obejmują wszystkie istotne aspekty roli nowego pracownika.",
            "Koordynuj logistykę nadchodzącego wydarzenia firmowego, takiego jak konferencja czy wyjazd integracyjny zespołu. Zabezpiecz lokalizacje, catering, transport i wszelkie niezbędne wyposażenie.",
            "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu, uwzględniając opinię użytkowników, trendy rynkowe i postęp technologiczny. Przedstaw pomysły zespołowi rozwoju produktu w celu oceny.",
            "Optymalizuj SEO strony internetowej, aby poprawić pozycje w wynikach wyszukiwania i zwiększyć ruch organiczny. Przeprowadź badania słów kluczowych, aktualizuj meta tagi i wdroż na stronie najlepsze praktyki SEO.",
            "Przeprowadź oceny wyników pracowników, dostarczając konstruktywną opinię i wyznaczając cele rozwoju osobistego i zawodowego. Doceniaj i nagradzaj wyjątkowe osiągnięcia.",
            "Twórz interaktywne prototypy dla nowej aplikacji, uwzględniając przyjazny dla użytkownika design i intuicyjną nawigację. Testuj prototypy z potencjalnymi użytkownikami, aby uzyskać opinię i walidację."
    );

    public static List<Task> generateTasks(int count) {
        List<Task> tasks = new ArrayList<>();
        Random random = new Random();

        for (int i = 0; i < count; i++) {
            String title = titles.get(random.nextInt(titles.size()));
            String description = descriptions.get(random.nextInt(descriptions.size()));
            Priority priority = Priority.values()[random.nextInt(Priority.values().length)];
            tasks.add(new Task(title, description, priority));
        }

        return tasks;
    }
}

## Baza danych

Priorytet zdefiniujemy w klasie `enum`

In [ ]:
public enum Priority {
    WYSOKI,
    NORMALNY,
    NISKI,
    WYKONANY
}

Dodajmy interfejs `dao` zawierający definicje operacji na bazie danych. 

In [ ]:
@Dao
public interface TaskDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertTask(Task task);

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insertAllTasks(List<Task> tasks);

    @Query("SELECT * FROM task_table ORDER BY id ASC")
    LiveData<List<Task>> getTasks();

    @Delete
    void deleteTask(Task task);

    @Update
    void updateTask(Task task);

    @Query("DELETE FROM task_table")
    void deleteAll();
}

Dodajmy repozytorium z odpowiednimi metodami.

In [ ]:
public class TaskRepository {
    private TaskDatabase db;
    private TaskDao dao;

    public TaskRepository(Application application) {
        db = TaskDatabase.getDatabase(application);
        dao = db.taskDao();
    }

    public LiveData<List<Task>> getTasks() {
        return dao.getTasks();
    }

    public void insertTask(Task task) {
        AppExecutors.getInstance().diskIO().execute(() -> dao.insertTask(task));
    }

    public void insertAllTasks(List<Task> tasks) {
        AppExecutors.getInstance().diskIO().execute(() -> dao.insertAllTasks(tasks));
    }

    public void deleteTask(Task task) {
        AppExecutors.getInstance().diskIO().execute(() -> dao.deleteTask(task));
    }

    public void updateTask(Task task) {
        AppExecutors.getInstance().diskIO().execute(() -> dao.updateTask(task));
    }

    public void deleteAll() {
        AppExecutors.getInstance().diskIO().execute(() -> dao.deleteAll());
    }
}

Następnie dodajmy `ViewModel`, który zarządza danymi związanymi z zadaniami, interakcjami z repozytorium (dostęp do danych) oraz logiką biznesową związaną z priorytetami i stanem zadań.

- tasksState - `LiveData` z listą zadań.
- Inicjalizacja bazy danych z danymi przykładowymi - w konstruktorze, `reinitializeDatabaseWithDummyData()` jest wywołane w celu dodania przykładowych danych do bazy danych poprzez usuwanie istniejących zadań i dodawanie nowych, pochodzących z `DataProvider.tasks`.
- Operacje CRUD na bazie danych - metody `addAll`, `deleteAll`, `updateTask`, `insertTask` i `deleteTask` są wykorzystywane do interakcji z bazą danych poprzez repozytorium. Wszystkie działania są wykonywane asynchroniczne i bezpieczne w kontekście wątków. Klasa `AppExecutors` zawiera metody umożliwiające asynchroniczne przetwarzanie.
- Logika priorytetów i stanu zadań - pozostała część kodu zawiera logikę związaną z priorytetami i stanem zadań. Metody `getTaskColor`, `increasePriority`, `decreasePriority` i `donePriority` zmieniają priorytet oraz stan zadań w zależności od ich obecnych wartości.
- Funkcje pomocnicze - `getTask(int id)` pozwala na pobranie konkretnego zadania o podanym `id`. `getTaskColor(Task task)` zwraca kolor w zależności od priorytetu zadania.

In [ ]:
public class AppExecutors {
    private static final Object LOCK = new Object();
    private static AppExecutors instance;
    private final Executor dbIO;

    private AppExecutors(Executor dbIO) {
        this.dbIO = dbIO;
    }

    public static AppExecutors getInstance() {
        if (instance == null) {
            synchronized (LOCK) {
                instance = new AppExecutors(Executors.newSingleThreadExecutor());
            }
        }
        return instance;
    }

    public Executor diskIO() {
        return dbIO;
    }
}

In [ ]:
public class TaskViewModel extends AndroidViewModel {

    private final TaskRepository repository;

    public final LiveData<List<Task>> tasksState;

    public TaskViewModel(Application application) {
        super(application);
        repository = new TaskRepository(application);
        tasksState = repository.getTasks();
        reinitializeDatabaseWithDummyData();
    }

    private void reinitializeDatabaseWithDummyData() {
        deleteAll();
        addAll(DataProvider.generateTasks(10));
    }

    private void addAll(List<Task> tasks) {
        for (Task task : tasks) {
            addTask(task);
        }
    }

    private void deleteAll() {
        repository.deleteAll();
    }

    public void updateTask(Task task) {
        repository.updateTask(task);
    }

    public Task getTask(int id) {
        List<Task> tasks = tasksState.getValue();
        return tasks != null ? tasks.stream()
                .filter(task -> task.getId() == id)
                .findFirst()
                .orElse(new Task("", "")) : new Task("", "");
    }

    public void addTask(Task task) {
        repository.insertTask(task);
    }

    public void deleteTask(Task task) {
        repository.deleteTask(task);
    }

    public Color getTaskColor(Task task) {
        switch (task.getPriority()) {
            case WYSOKI:
                return Color.valueOf(Color.RED);
            case NISKI:
                return Color.valueOf(Color.BLUE);
            case NORMALNY:
                return Color.valueOf(Color.GREEN);
            default:
                return Color.valueOf(128f, 203f, 196f);
        }
    }

    public void increasePriority(Task task) {
        Priority nextPriority;
        switch (task.getPriority()) {
            case WYSOKI:
            case NORMALNY:
                nextPriority = Priority.WYSOKI;
                break;
            case NISKI:
                nextPriority = Priority.NORMALNY;
                break;
            default:
                nextPriority = Priority.WYKONANY;
        }
        task.setPriority(nextPriority);
        updateTask(task);
    }

    public void decreasePriority(Task task) {
        Priority nextPriority;
        switch (task.getPriority()) {
            case WYSOKI:
                nextPriority = Priority.NORMALNY;
                break;
            case NISKI:
            case NORMALNY:
                nextPriority = Priority.NISKI;
                break;
            default:
                nextPriority = Priority.WYKONANY;
        }
        task.setPriority(nextPriority);
        updateTask(task);
    }

    public void donePriority(Task task) {
        task.setDone(true);
        task.setPriority(Priority.WYKONANY);
        updateTask(task);    }
}


Aplikacja posiada trzy ekrany, głównym jest lista wszystkich zadań. W nawigacji przejdziemy bezargumentowo na ekran dodania nowego zadania. Ekran aktualizacji zadania będzie wymagał przekazania `id` zadania, dzięki któremu załadujemy odpowiednie dane z listy. Sam `ViewModel` będzie **współdzielony** przez wszystkie ekrany, osiągniemy to poprzez jego zainicjowanie w komponencie `Navigation` i przekazanie tej instancji przez parametr do komponentów reprezentujących ekrany.

## Nawigacja

Dodajmy nawigację.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/taskListFragment">

    <fragment
        android:id="@+id/taskListFragment"
        android:name="com.example.prioritezemejava.ui.fragments.list.ListFragment"
        android:label="fragment_task_list"
        tools:layout="@layout/fragment_list" >
        <action
            android:id="@+id/action_taskListFragment_to_taskAddFragment"
            app:destination="@id/taskAddFragment" />
        <action
            android:id="@+id/action_taskListFragment_to_taskUpdateFragment"
            app:destination="@id/taskUpdateFragment" >
            <argument
                android:name="task_id"
                app:argType="integer"
                android:defaultValue="0" />
        </action>
    </fragment>
    <fragment
        android:id="@+id/taskAddFragment"
        android:name="com.example.prioritezemejava.ui.fragments.add.AddFragment"
        android:label="fragment_task_add"
        tools:layout="@layout/fragment_add" >
        <action
            android:id="@+id/action_taskAddFragment_to_taskListFragment"
            app:destination="@id/taskListFragment" />
    </fragment>
    <fragment
        android:id="@+id/taskUpdateFragment"
        android:name="com.example.prioritezemejava.ui.fragments.edit.UpdateFragment"
        android:label="fragment_task_update"
        tools:layout="@layout/fragment_update" >
        <action
            android:id="@+id/action_taskUpdateFragment_to_taskListFragment"
            app:destination="@id/taskListFragment" />
    </fragment>
</navigation>

Na Fragment przznaczony do aktualizacji zadania będziemy przekazywać argument - `id` zadania, więc do `action` dodajemy odpowiedni `argument`

```xml
<action
    android:id="@+id/action_taskListFragment_to_taskUpdateFragment"
    app:destination="@id/taskUpdateFragment" >
    <argument
        android:name="task_id"
        app:argType="integer"
        android:defaultValue="0" />
</action>
```

Będziemy wykorzystywać **współdzielony** `ViewModel`, polega to na zainicjowaniu instancji w aktywności hostującej i wykorzystanie odpowiedniego delegatu (`by activityViewModels`) we fragmentach. W ten sposób wszystkie fragmenty będą posiadały dokładnie tą samą instancję `ViewModel`.

In [ ]:
public class MainActivity extends AppCompatActivity {

    public TaskViewModel viewModel;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        viewModel = new ViewModelProvider(this).get(TaskViewModel.class);
    }
}

## Ekrany

Przejdźmy do zaprojektowania layoutu fragmentu z listą.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:layout_margin="4dp"
    tools:context=".ui.fragments.list.TaskListFragment">

    <com.google.android.material.floatingactionbutton.FloatingActionButton
        android:id="@+id/addFab"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="24dp"
        android:layout_marginBottom="36dp"
        android:clickable="true"
        android:src="@drawable/ic_add"
        style="@style/ShapeAppearanceOverlay.Material3.FloatingActionButton"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:contentDescription="add" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recycler"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="8dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Oprócz `RecyclerView` wykorzystujemy tutaj `FAB` (*Floating Action Button*). FAB jest przyciskiem umieszczonym w obszarze interfejsu, który zazwyczaj znajduje się na dolnej części ekranu. Jego celem jest dostarczenie szybkiego dostępu do kluczowych akcji lub funkcji aplikacji. Tutaj wykorzystamy ten przycisk aby umożliwić użytkownikowi przejście na ekran dodania zadania.

Będziemy wykorzystywać `RecyclerView`, więc potrzebujemy również layout dla pojedynczego elementu listy.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<com.google.android.material.card.MaterialCardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="wrap_content"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    app:strokeColor="@color/black"
    android:background="#FF6200EE"
    app:cardCornerRadius="8dp">

    <LinearLayout
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:orientation="vertical"
        >

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="horizontal"
            >

            <TextView
                android:id="@+id/title"
                android:layout_width="0dp"
                android:layout_height="wrap_content"
                android:layout_weight="1"
                android:background="#B093E6"
                android:gravity="center"
                android:padding="4dp"
                android:text="Przeprowadź Ankietę Użytkowników"
                android:textColor="@color/black"
                android:textSize="16sp" />

            <ImageView
                android:id="@+id/priorityIcon"
                android:layout_width="wrap_content"
                android:layout_height="match_parent"
                android:layout_gravity="center"
                android:scaleType="center"
                android:paddingEnd="4dp"
                android:background="#B093E6"
                android:scaleX="1"
                android:scaleY="1"
                android:src="@drawable/ic_circle"
                android:contentDescription="TODO" />
        </LinearLayout>

        <TextView
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:text="Optymalizuj SEO strony internetowej, ..."
            android:textSize="14sp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="vertical">
            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="horizontal">

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/priorityUp"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="8dp"
                    android:layout_marginEnd="4dp"
                    app:icon="@drawable/ic_up"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/done"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="4dp"
                    app:icon="@drawable/ic_done"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/priorityDown"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="4dp"
                    android:layout_height="wrap_content"
                    app:icon="@drawable/ic_down"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/delete"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="8dp"
                    android:layout_height="wrap_content"
                    app:icon="@drawable/ic_delete"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>
            </LinearLayout>

            <com.google.android.material.button.MaterialButton
                android:id="@+id/edit"
                style="?attr/materialButtonOutlinedStyle"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="8dp"
                android:layout_marginEnd="8dp"
                app:icon="@drawable/ic_edit"
                app:iconGravity="textStart"
                app:iconPadding="0dp" />

        </LinearLayout>

    </LinearLayout>
</com.google.android.material.card.MaterialCardView>

`ImageView` jest elementem, który zostanie wykorzystany w celu pokazania priorytetu zadania. Sam priorytet będziemy reprezentować za pomocą koloru. W tradycyjnym projektowaniu interfejsu (za pomocą plików xml i widoków) nie występuje `IconButton`, więc aby dodać przycisk tylko z ikoną (bez tekstu) musimy zasotować prosty trik.

```xml
app:icon="@drawable/ic_down"
app:iconGravity="textStart"
app:iconPadding="0dp"
```

Ustawiamy właściwość `iconGravity` na `textStart` aby umieścić ikonę przed tekstem, następnie ustawiamy `iconPadding` na `0dp`. W ten sposób, przy braku tekstu, osiągniemy przycisk z wyśrodkowaną ikoną.

Przejdźmy do implementacji klas wymaganych do obsługi `RecyclerView`.

In [ ]:
public class TaskViewHolder extends RecyclerView.ViewHolder {

    private final RvItemBinding binding;

    private final OnPriorityUpClickListener onPriorityUp;
    private final OnPriorityDownClickListener onPriorityDown;
    private final OnTaskDoneClickListener onTaskDone;
    private final OnDeleteClickListener onDelete;
    private final OnEditClickListener onEdit;

    public TaskViewHolder(RvItemBinding binding,
                          OnPriorityUpClickListener onPriorityUp,
                          OnPriorityDownClickListener onPriorityDown,
                          OnTaskDoneClickListener onTaskDone,
                          OnDeleteClickListener onDelete,
                          OnEditClickListener onEdit) {
        super(binding.getRoot());
        this.binding = binding;

        this.onPriorityUp = onPriorityUp;
        this.onPriorityDown = onPriorityDown;
        this.onTaskDone = onTaskDone;
        this.onDelete = onDelete;
        this.onEdit = onEdit;
    }

    public void bind(Task item) {
        binding.title.setText(item.getTitle());
        binding.description.setText(item.getDescription());
        Color color = priorityColor(item);
        binding.priorityIcon.setColorFilter(Color.rgb(color.red(), color.green(), color.blue()), PorterDuff.Mode.SRC_IN);

        binding.priorityUp.setOnClickListener(v -> onPriorityUp.onPriorityUp(item));
        binding.priorityDown.setOnClickListener(v -> onPriorityDown.onPriorityDown(item));
        binding.done.setOnClickListener(v -> onTaskDone.onTaskDone(item));
        binding.delete.setOnClickListener(v -> onDelete.onDelete(item));
        binding.edit.setOnClickListener(v -> onEdit.onEdit(item.getId()));
    }

    private Color priorityColor(Task task) {
        switch (task.getPriority()) {
            case WYSOKI:
                return Color.valueOf(Color.RED);
            case NISKI:
                return Color.valueOf(Color.BLUE);
            case NORMALNY:
                return Color.valueOf(Color.GREEN);
            default:
                return Color.valueOf(128f, 203f, 196f);
        }
    }

    public interface OnPriorityUpClickListener {
        void onPriorityUp(Task task);
    }

    public interface OnPriorityDownClickListener {
        void onPriorityDown(Task task);
    }

    public interface OnTaskDoneClickListener {
        void onTaskDone(Task task);
    }

    public interface OnDeleteClickListener {
        void onDelete(Task task);
    }

    public interface OnEditClickListener {
        void onEdit(int position);
    }
}


- Konstruktor klasy `TaskViewHolder`:
    - `binding` Przekazywany jest obiekt typu `RvItemBinding`, który reprezentuje powiązanie widoku elementu listy z odpowiednimi komponentami.
    - `onPriorityUp` Obiekt o typie interfejsu zawierający jedną metodę wywoływaną po kliknięciu przycisku *Priority Up* w widoku elementu. Przekazywana jest tutaj funkcja lambda przyjmująca obiekt typu `Task`.
    - `onPriorityDown` Obiekt o typie interfejsu zawierający jedną metodę wywoływaną po kliknięciu przycisku *Priority Down* w widoku elementu.
    - `onTaskDone` Obiekt o typie interfejsu zawierający jedną metodę wywoływaną po kliknięciu przycisku *Done* w widoku elementu.
    - `onDelete` Obiekt o typie interfejsu zawierający jedną metodę wywoływaną po kliknięciu przycisku *Delete* w widoku elementu.
    - `onEdit` Obiekt o typie interfejsu zawierający jedną metodę wywoływaną po kliknięciu przycisku *Edit* w widoku elementu. Przekazywana jest tutaj funkcja lambda przyjmująca identyfikator (`Int`) zadania.
    

- Metoda `bind(Task item)`:
    Metoda ta jest używana do wiązania danych przekazanych do widoku elementu.
    W tej metodzie następuje ustawienie tytułu i opisu zadania z obiektu `item` do odpowiednich komponentów interfejsu użytkownika.
    Kolor ikony priorytetu jest ustawiany na podstawie funkcji `priorityColor(item)` i przekazywany za pomocą `setColorFilter`.
    
- Prywatna metoda `priorityColor(Task task): Color`:
    Ta metoda przyjmuje obiekt typu `Task` i zwraca kolor, który reprezentuje priorytet tego zadania.
    W zależności od wartości priorytetu zadania, metoda zwraca odpowiedni kolor (np. czerwony dla priorytetu *Wysoki*, niebieski dla *Niski* itp.).

In [ ]:
public class TaskComparator extends DiffUtil.ItemCallback<Task> {
    @Override
    public boolean areItemsTheSame(@NonNull Task oldItem, @NonNull Task newItem) {
        return newItem == oldItem;
    }

    @Override
    public boolean areContentsTheSame(@NonNull Task oldItem, Task newItem) {
        return newItem.equals(oldItem);
    }
}

Klasa `TaskComparator` dziedziczy po klasie `DiffUtil.ItemCallback<Task>()` i jest wykorzystywana w celu porównywania i obliczania różnic między dwoma listami obiektów typu `Task`. W kontekście komponentu `RecyclerView`, `DiffUtil` pomaga zoptymalizować wydajność odświeżania listy poprzez identyfikację tylko tych elementów, które faktycznie uległy zmianie.

In [ ]:
public class TaskAdapter extends ListAdapter<Task, TaskViewHolder> {

    private final TaskViewHolder.OnPriorityUpClickListener onPriorityUp;
    private final TaskViewHolder.OnPriorityDownClickListener onPriorityDown;
    private final TaskViewHolder.OnTaskDoneClickListener onTaskDone;
    private final TaskViewHolder.OnDeleteClickListener onDelete;
    private final TaskViewHolder.OnEditClickListener onEdit;

    protected TaskAdapter(TaskComparator taskComparator,
                          TaskViewHolder.OnPriorityUpClickListener onPriorityUp,
                          TaskViewHolder.OnPriorityDownClickListener onPriorityDown,
                          TaskViewHolder.OnTaskDoneClickListener onTaskDone,
                          TaskViewHolder.OnDeleteClickListener onDelete,
                          TaskViewHolder.OnEditClickListener onEdit) {
        super(taskComparator);
        this.onPriorityUp = onPriorityUp;
        this.onPriorityDown = onPriorityDown;
        this.onTaskDone = onTaskDone;
        this.onDelete = onDelete;
        this.onEdit = onEdit;
    }

    @NonNull
    @Override
    public TaskViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new TaskViewHolder(
                RvItemBinding.inflate(
                        LayoutInflater.from(parent.getContext()), parent, false
                ),
                onPriorityUp,
                onPriorityDown,
                onTaskDone,
                onDelete,
                onEdit
        );
    }

    @Override
    public void onBindViewHolder(TaskViewHolder holder, int position) {
        Task item = getItem(position);
        holder.bind(item);
    }
}

- Konstruktor klasy `TaskAdapter`:
    - `taskComparator` Przekazywany jest obiekt klasy `TaskComparator`, który będzie wykorzystywany do porównywania obiektów w celu określenia zmian w liście.
    - Pozostałe parametry to lambdy i funkcje, które zostaną wywołane w odpowiedzi na interakcje użytkownika z elementami listy.
- Metoda `onCreateViewHolder`:
    Ta metoda tworzy nowy obiekt TaskViewHolder poprzez opakowanie widoku z layoutu RvItemBinding za pomocą LayoutInflater.
    Przekazywane są również funkcje i lambdy obsługujące interakcje związane z priorytetem, oznaczaniem jako zrobione, usuwaniem i edytowaniem zadania.
- Metoda `onBindViewHolder`:
    Ta metoda wiąże obiekt `TaskViewHolder` z odpowiednim obiektem `Task` z listy.
    Uzyskiwany jest element z listy za pomocą `getItem(position)`, a następnie przekazywany do metody `bind(item)`, aby uzupełnić widok danymi.

Przejdźmy do implementacji samego fragmentu

In [ ]:
public class ListFragment extends Fragment {
    private FragmentListBinding binding;
    private TaskViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentListBinding.inflate(inflater, container, false);
        viewModel = new ViewModelProvider(requireActivity()).get(TaskViewModel.class);

        TaskAdapter taskAdapter = new TaskAdapter(
                new TaskComparator(),
                task -> viewModel.increasePriority(task),
                task -> viewModel.decreasePriority(task),
                task -> viewModel.donePriority(task),
                task -> viewModel.deleteTask(task),
                taskId -> {
                    ListFragmentDirections.ActionTaskListFragmentToTaskUpdateFragment action =
                            ListFragmentDirections.actionTaskListFragmentToTaskUpdateFragment();
                    action.setTaskId(taskId);
                    Navigation.findNavController(requireView()).navigate(action);
                }
        );

        viewModel.tasksState.observe(getViewLifecycleOwner(), taskAdapter::submitList);

        binding.recycler.setAdapter(taskAdapter);
        binding.recycler.setLayoutManager(new StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL));
        binding.recycler.setItemAnimator(null);

        binding.addFab.setOnClickListener(v -> {
            Navigation.findNavController(requireView())
                    .navigate(ListFragmentDirections.actionTaskListFragmentToTaskAddFragment());
        });

        return binding.getRoot();
    }
}

Przejdźmy przez kod krok po kroku

In [ ]:
viewModel = new ViewModelProvider(requireActivity()).get(TaskViewModel.class);

- `ViewModel`: jest komponentem, który przechowuje i dostarcza dane związane z interfejsem użytkownika, takie jak stan, które nie powinno być przechowywane bezpośrednio w fragmencie czy aktywności, aby przetrwać zmiany konfiguracji (np. obrót ekranu).
- `ViewModel` *Sharing*: `ViewModelProvider(requireActivity()).get(TaskViewModel.class)` to mechanizm, który umożliwia dzielenie tego samego `ViewModel`u między różnymi fragmentami (lub innymi komponentami) znajdującymi się w tym samym zakresie, np. w obrębie jednej aktywności. Oznacza to, że fragment `TaskListFragment` będzie używać `ViewModel`u, który jest dostępny w zakresie aktywności hostującej.

In [ ]:
TaskAdapter taskAdapter = new TaskAdapter(
        new TaskComparator(),
        task -> viewModel.increasePriority(task),
        task -> viewModel.decreasePriority(task),
        task -> viewModel.donePriority(task),
        task -> viewModel.deleteTask(task),
        taskId -> {
            ListFragmentDirections.ActionTaskListFragmentToTaskUpdateFragment action =
                    ListFragmentDirections.actionTaskListFragmentToTaskUpdateFragment();
            action.setTaskId(taskId);
            Navigation.findNavController(requireView()).navigate(action);
        }
);

- `task -> viewModel.increasePriority(task)`: To jest funkcja, która zostanie przekazana do adaptera i będzie wywoływana, gdy użytkownik kliknie przycisk zwiększający priorytet w widoku elementu listy. Wewnątrz tej lambdy jest wywoływana funkcja `increasePriority` z `viewModel`, a jako argument przekazywane jest obiekt `Task`, który został kliknięty.
- `task -> viewModel.decreasePriority(task)`: To jest podobna funkcja lambda do poprzedniej, tylko w tym przypadku jest przypisana akcja spadku priorytetu. 
- `task -> viewModel.donePriority(task)`: zostanie wywołana po kliknięciu przycisku *Done* w widoku elementu listy. Wewnątrz tej lambdy jest wywoływana funkcja `donePriority` z `viewModel`.
- `task -> viewModel.deleteTask(task)`: Podobnie jak poprzednie, ta funkcja lambda obsługuje kliknięcie przycisku *Delete* w widoku elementu listy i wywołuje funkcję `deleteTask` z `viewModel`.
- `taskId -> {...}`: Tworzony jest obiekt *akcji*, która nawiguje użytkownika do fragmentu aktualizacji zadania z przekazanym identyfikatorem zadania. Następnie jest wywołana nawigacja za pomocą `Navigation.findNavController(requireView()).navigate(action)`.

In [ ]:
viewModel.tasksState.observe(getViewLifecycleOwner(), taskAdapter::submitList);

- `viewModel.tasksState` to `LiveData` reprezentujący stan listy zadań.
- W momencie, gdy dane w `viewModel.tasksState` się zmieniają, blok kodu wewnątrz `observe` zostaje wywołany.
- Wewnątrz tego bloku, otrzymana lista zadań (`taskList`) jest przekazywana do adaptera poprzez `taskAdapter.submitList(taskList)`, co prowadzi do aktualizacji widoku listy zadań w interfejsie użytkownika.

In [ ]:
binding.recycler.setAdapter(taskAdapter);
binding.recycler.setLayoutManager(new StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL));
binding.recycler.setItemAnimator(null);

- `layoutManager = new StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL)`: Ten wiersz kodu ustawia menedżer układu dla `RecyclerView` na `StaggeredGridLayoutManager`.
- `StaggeredGridLayoutManager` to menedżer układu, który tworzy układ siatki, gdzie elementy w rzędzie mogą mieć różne wysokości (niezależnie od siebie). Dzięki temu uzyskujemy efekt kafelków o nieregularnych wysokościach. W tym przypadku, tworzony jest układ dwukolumnowy (2 oznacza dwie kolumny) z orientacją pionową (`StaggeredGridLayoutManager.VERTICAL`).
- `binding.recycler.setItemAnimator(null)`: Ten wiersz kodu wyłącza domyślny animowany efekt dla zmian elementów w `RecyclerView`.

><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" />

Następnie dodajmy ekran nowego zadania.

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    android:layout_margin="4dp"
    tools:context=".ui.fragments.add.TaskAddFragment">

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/titleInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:hint="Tytuł">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/title"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textShortMessage"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/descriptionInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:hint="Opis">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textMultiLine"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">

        <RadioGroup
            android:id="@+id/radio"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="vertical">

            <RadioButton
                android:id="@+id/radioHigh"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="WYSOKI" />

            <RadioButton
                android:id="@+id/radioNormal"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="NORMALNY" />

            <RadioButton
                android:id="@+id/radioLow"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="NISKI" />

            <RadioButton
                android:id="@+id/radioDone"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="WYKONANY" />
        </RadioGroup>
        </com.google.android.material.card.MaterialCardView>
        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="match_parent"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginEnd="8dp"
            android:layout_marginStart="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">
        <CheckBox
            android:id="@+id/checkbox"
            android:layout_width="match_parent"
            android:layout_gravity="center"
            android:layout_height="match_parent"
            android:text="Czy wykonane"
            android:textSize="20sp"/>
        </com.google.android.material.card.MaterialCardView>

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

    <com.google.android.material.button.MaterialButton
        android:id="@+id/save"
        android:layout_width="0dp"
        android:layout_weight=".5"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="1dp"
        android:text="Zapisz"/>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/back"
        android:layout_width="0dp"
        android:layout_weight=".5"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="1dp"
        android:layout_marginEnd="4dp"
        style="?attr/materialButtonOutlinedStyle"
        android:text="Wróć"/>
    </LinearLayout>


</LinearLayout>

- `TextInputLayout` dla pola *Tytuł*: Widok obsługujący pole tekstowe dla tytułu zadania. Korzysta z komponentu `TextInputLayout`, który dostarcza etykietę i efekty wizualne do pola tekstowego.
- `TextInputLayout dla pola *Opis*`: Podobnie jak wyżej, ale to pole obsługuje dłuższy tekst, używając typu `textMultiLine` jako rodzaj wprowadzanych danych.
- Grupa `RadioButton`ów: Znajduje się wewnątrz karty (`MaterialCardView`). *Pozwala na wybór priorytetu zadania spośród czterech opcji: Wysoki, Normalny, Niski, Wykonany*.
- `CheckBox` *Czy wykonane*: Druga karta zawiera `CheckBox` umożliwiający zaznaczenie zadania jako wykonane lub nie.
- Przyciski *Zapisz* i *Wróć*: Przyciski do zapisania nowego zadania lub powrotu do poprzedniego ekranu. Ustawione są w linii poziomej, przy użyciu `LinearLayout`.

Przejdźmy do implementacji fragmentu

In [ ]:
public class AddFragment extends Fragment {
    private FragmentAddBinding binding;
    private TaskViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentAddBinding.inflate(inflater);

        viewModel = new ViewModelProvider(requireActivity()).get(TaskViewModel.class);

        boolean[] enabled = {true};

        binding.radioNormal.setChecked(true);
        binding.checkbox.setOnClickListener(v -> {
            enabled[0] = !enabled[0];
            changeRadioEnabled(enabled[0]);
            if (!enabled[0]) {
                binding.radioDone.setChecked(true);
            }
        });

        binding.save.setOnClickListener(v -> {
            String title = "";
            String description = "";
            if (binding.title.getText() != null && binding.description.getText() != null) {
                title = binding.title.getText().toString();
                description = binding.description.getText().toString();
            }

            if (!title.isEmpty() || !description.isEmpty()) {
                Priority priority = getPriority();
                Task task = new Task(title, description, binding.checkbox.isChecked(), priority);
                viewModel.addTask(task);

                Navigation
                        .findNavController(requireView())
                        .navigate(AddFragmentDirections.actionTaskAddFragmentToTaskListFragment());
            }
        });

        binding.back.setOnClickListener(v -> Navigation
                .findNavController(requireView())
                .navigate(AddFragmentDirections.actionTaskAddFragmentToTaskListFragment()));

        return binding.getRoot();
    }

    private Priority getPriority() {
        int checkedRadioButtonId = binding.radio.getCheckedRadioButtonId();
        if (checkedRadioButtonId == binding.radioHigh.getId()) {
            return Priority.WYSOKI;
        } else if (checkedRadioButtonId == binding.radioNormal.getId()) {
            return Priority.NORMALNY;
        } else if (checkedRadioButtonId == binding.radioLow.getId()) {
            return Priority.NISKI;
        } else {
            return Priority.WYKONANY;
        }
    }

    private void changeRadioEnabled(boolean enabled){
        binding.radioDone.setEnabled(enabled);
        binding.radioNormal.setEnabled(enabled);
        binding.radioHigh.setEnabled(enabled);
        binding.radioLow.setEnabled(enabled);
    }
}

Zmienna `enabled` kontroluje dostępność opcji wyboru priorytetu. Wybrany priorytet inny niż *Wykonany* jest dostępny, a po zaznaczeniu `CheckBoxa`, przestają być dostępne i automatycznie wybierany jest *Wykonany*. 

In [ ]:
binding.checkbox.setOnClickListener(v -> {
    enabled[0] = !enabled[0];
    changeRadioEnabled(enabled[0]);
    if (!enabled[0]) {
        binding.radioDone.setChecked(true);
    }
});

In [ ]:
private void changeRadioEnabled(boolean enabled){
    binding.radioDone.setEnabled(enabled);
    binding.radioNormal.setEnabled(enabled);
    binding.radioHigh.setEnabled(enabled);
    binding.radioLow.setEnabled(enabled);
}

Dla każdego `RadioButtona`, właściwość `isEnabled` jest ustawiana na wartość zmiennej `enabled`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" />

Ostatnim ekranem będzie ekran aktualizacji zadania, będzie on podobny do ekranu dodawania zadania.

Layout jest bardzo podobny do layoutu ekranu dodawania zadania.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    tools:context=".ui.fragments.add.TaskAddFragment">

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/titleInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:hint="Tytuł">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/title"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textShortMessage"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/descriptionInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:hint="Opis">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textMultiLine"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">

            <RadioGroup
                android:id="@+id/radio"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="vertical">

                <RadioButton
                    android:id="@+id/radioHigh"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="WYSOKI" />

                <RadioButton
                    android:id="@+id/radioNormal"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="NORMALNY" />

                <RadioButton
                    android:id="@+id/radioLow"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="NISKI" />

                <RadioButton
                    android:id="@+id/radioDone"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="WYKONANY" />
            </RadioGroup>
        </com.google.android.material.card.MaterialCardView>
        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="match_parent"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginEnd="8dp"
            android:layout_marginStart="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">
            <CheckBox
                android:id="@+id/checkbox"
                android:layout_width="match_parent"
                android:layout_gravity="center"
                android:layout_height="match_parent"
                android:text="Czy wykonane"
                android:textSize="20sp"/>
        </com.google.android.material.card.MaterialCardView>

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.button.MaterialButton
            android:id="@+id/update"
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            android:textSize="20sp"
            android:layout_marginStart="4dp"
            android:layout_marginEnd="1dp"
            android:text="Aktualizuj"/>

        <com.google.android.material.button.MaterialButton
            android:id="@+id/delete"
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            android:textSize="20sp"
            android:layout_marginStart="1dp"
            android:layout_marginEnd="4dp"
            android:text="Usuń"/>
    </LinearLayout>
    <com.google.android.material.button.MaterialButton
        android:id="@+id/back"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="1dp"
        android:layout_marginEnd="4dp"
        style="?attr/materialButtonOutlinedStyle"
        android:text="Wróć"/>


</LinearLayout>

In [ ]:
public class UpdateFragment extends Fragment {

    private FragmentUpdateBinding binding;
    private TaskViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentUpdateBinding.inflate(inflater);

        viewModel = new ViewModelProvider(requireActivity()).get(TaskViewModel.class);

        boolean[] enabled = {true};

        int id = getArguments().getInt("task_id", 0);

        Task task = viewModel.getTask(id);

        binding.title.setText(task.getTitle());
        binding.description.setText(task.getDescription());
        setRadio(task.getPriority());
        binding.checkbox.setChecked(task.isDone());

        binding.checkbox.setOnClickListener(v -> {
            enabled[0] = !enabled[0];
            changeRadioEnabled(enabled[0]);
            if (!enabled[0]) {
                binding.radioDone.setChecked(true);
            }
        });

        binding.update.setOnClickListener(v -> {
            String title = binding.title.getText().toString();
            String description = binding.description.getText().toString();

            if (!title.isEmpty() || !description.isEmpty()) {
                Priority priority = getPriority();
                task.setTitle(title);
                task.setDescription(description);
                task.setDone(binding.checkbox.isChecked());
                task.setPriority(priority);
                viewModel.updateTask(task);

                Navigation
                        .findNavController(requireView())
                        .navigate(UpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment());
            }
        });

        binding.back.setOnClickListener(v -> Navigation
                .findNavController(requireView())
                .navigate(UpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment()));

        binding.delete.setOnClickListener(v -> {
            viewModel.deleteTask(task);

            Navigation
                    .findNavController(requireView())
                    .navigate(UpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment());
        });

        return binding.getRoot();
    }

    private void setRadio(Priority priority) {
        if (priority == Priority.WYSOKI) {
            binding.radioHigh.setChecked(true);
        } else if (priority == Priority.NORMALNY) {
            binding.radioNormal.setChecked(true);
        } else if (priority == Priority.NISKI) {
            binding.radioLow.setChecked(true);
        } else {
            binding.radioDone.setChecked(true);
        }
    }

    private Priority getPriority() {
        int checkedRadioButtonId = binding.radio.getCheckedRadioButtonId();
        if (checkedRadioButtonId == binding.radioHigh.getId()) {
            return Priority.WYSOKI;
        } else if (checkedRadioButtonId == binding.radioNormal.getId()) {
            return Priority.NORMALNY;
        } else if (checkedRadioButtonId == binding.radioLow.getId()) {
            return Priority.NISKI;
        } else {
            return Priority.WYKONANY;
        }
    }

    private void changeRadioEnabled(boolean enabled){
        binding.radioDone.setEnabled(enabled);
        binding.radioNormal.setEnabled(enabled);
        binding.radioHigh.setEnabled(enabled);
        binding.radioLow.setEnabled(enabled);
    }
}

Implementacja `UpdateTaskFragment` jest podobna do `AddTaskFragment`, różnicą jest odebranie identyfikatora zadania przekazanego przez `Navigation` z fragmentu `TaskListFragment` i pobranie odpowiadniego zadania z listy dostępnej w `ViewModel`

In [ ]:
int id = getArguments().getInt("task_id", 0);

Task task = viewModel.getTask(id);

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" />